# Projeto DATASUS - Daniel Gama

**Fonte: Datasus** - https://www.kaggle.com/datasets/psicodata/dados-de-suicdio-no-brasil-2014-a-2018

Foco: Carregar, inspecionar, tratar valores ausentes simples, corrigir tipos de dados e carregar no banco de dados `Postgres`.

**Contexto do projeto**: Meu objetivo é limpar e preparar uma base de dados relacionados a suicídios para criar um relatório demográfico.

## Passo 1: Configurando Ambiente

In [2]:
# Instalar as ferramentas necessárias para trabalhar com dados do tipo .csv e banco de dados
%pip install pandas sqlalchemy psycopg2-binary sqlalchemy-utils -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists,create_database
import psycopg2
import warnings

warnings.filterwarnings('ignore')

## Passo 2: E (Extract) - Extração de Dados

In [42]:
#Carregando a Base de Dados Datasus
df_datasus = pd.read_csv('base-dados/datasus_suicidio_2014_2018.csv', encoding='ISO-8859-1', sep=',')

print('\nCinco primeias linhas da base:\n')
display(df_datasus.head())

print('\nDez linhas aleatórias:\n')
display(df_datasus.sample(10))


Cinco primeias linhas da base:



,estado,ano,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O,idade,mes
0,AC,2014,Suicídio,02-01-14,02-07-77,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Rio Branco,Domicílio,Não,X700,X700,37.0,1
1,AC,2014,Suicídio,23-01-14,30-07-66,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Rio Branco,Domicílio,Não,X800,X800,48.0,1
2,AC,2014,Suicídio,31-01-14,28-07-43,Masculino,Branca,NaN,NaN,0,Rio Branco,Domicílio,Não,X700,X700,71.0,1
3,AC,2014,Suicídio,05-02-14,04-07-99,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Epitaciolândia,Outros,Não,X700,X700,15.0,2
4,AC,2014,Suicídio,06-02-14,02-05-72,Masculino,Branca,NaN,NaN,0,Rio Branco,Outros,Não,X700,X700,42.0,2



Dez linhas aleatórias:



,estado,ano,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O,idade,mes
23487,DF,2016,Suicídio,19-06-16,27-09-88,Feminino,Parda,Solteiro,12 e mais,ESTUDANTE,Brasília,Domicílio,NaN,X700,X700,28.0,6
48312,GO,2018,Suicídio,14-06-18,09-05-75,Masculino,Parda,Casado,4 a 7 anos,DESOSSADOR,Goiânia,Outros,NaN,X700,X700,43.0,6
57335,SP,2018,Suicídio,07-06-18,09-12-74,Feminino,Branca,Solteiro,4 a 7 anos,EMPREGADO DOMESTICO FAXINEIRO,São Caetano do Sul,Outros,Não,X648,X648,44.0,6
30612,SC,2016,Suicídio,06-07-16,07-02-65,Masculino,Branca,Separado judicialmente,4 a 7 anos,PRODUTOR AGRICOLA POLIVALENTE,Santa Rosa de Lima,Domicílio,Não,X700,X700,51.0,7
11294,BA,2015,Suicídio,06-01-15,29-07-78,Masculino,Parda,Solteiro,12 e mais,ADVOGADO,Vitória da Conquista,Hospital,Sim,X619,X619,37.0,1
44046,SP,2017,NaN,06-08-17,11-05-02,Masculino,Branca,Solteiro,8 a 11 anos,ESTUDANTE,São Paulo,Outros,NaN,X800,Y299,15.0,8
37291,MG,2017,Suicídio,04-07-17,03-05-60,Masculino,Branca,Casado,4 a 7 anos,0,Santa Luzia,Domicílio,NaN,X700,X700,57.0,7
15244,PB,2015,Suicídio,07-01-15,09-07-75,Masculino,Parda,NaN,NaN,0,Triunfo,Domicílio,NaN,X700,X700,40.0,1
5297,PE,2014,Suicídio,23-04-14,16-08-81,Masculino,Parda,Solteiro,Nenhuma,TRABALHADOR VOLANTE DA AGRICULTURA,Gravatá,Domicílio,Não,X700,X700,33.0,4
32645,SP,2016,NaN,17-10-16,04-08-70,Feminino,Branca,União consensual,4 a 7 anos,DONA DE CASA,Bariri,Hospital,Sim,X680,X680,46.0,10


## Passo 3: T (Transform) - Transformação dos Dados

In [43]:
#Informações Gerais da Base:
print('\nInformações Gerais da Base de Dados:\n')
display(df_datasus.info())

#Descrição da Base:
print('\nDescrição da Base de dados:\n')
display(df_datasus.describe())

#Visualizando se existe campos nulos:
print('\nCampos nulos existêntes:\n')
display(df_datasus.isnull().sum())


Informações Gerais da Base de Dados:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58634 entries, 0 to 58633
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estado      58634 non-null  object 
 1   ano         58634 non-null  int64  
 2   CIRCOBITO   56606 non-null  object 
 3   DTOBITO     58634 non-null  object 
 4   DTNASC      58407 non-null  object 
 5   SEXO        58619 non-null  object 
 6   RACACOR     57298 non-null  object 
 7   ESTCIV      54305 non-null  object 
 8   ESC         45162 non-null  object 
 9   OCUP        58634 non-null  object 
 10  CODMUNRES   58634 non-null  object 
 11  LOCOCOR     58463 non-null  object 
 12  ASSISTMED   40872 non-null  object 
 13  CAUSABAS    58634 non-null  object 
 14  CAUSABAS_O  58544 non-null  object 
 15  idade       58407 non-null  float64
 16  mes         58634 non-null  int64  
dtypes: float64(1), int64(2), object(14)
memory usage: 7.6+ MB


None


Descrição da Base de dados:



,ano,idade,mes
count,58634.000000,58407.000000,58634.000000
mean,2016.093325,42.543308,6.585496
std,1.412665,17.378526,3.502479
min,2014.000000,0.000000,1.000000
25%,2015.000000,29.000000,3.000000
50%,2016.000000,40.000000,7.000000
75%,2017.000000,54.000000,10.000000
max,2018.000000,113.000000,12.000000



Campos nulos existêntes:



estado            0
ano               0
CIRCOBITO      2028
DTOBITO           0
DTNASC          227
SEXO             15
RACACOR        1336
ESTCIV         4329
ESC           13472
OCUP              0
CODMUNRES         0
LOCOCOR         171
ASSISTMED     17762
CAUSABAS          0
CAUSABAS_O       90
idade           227
mes               0
dtype: int64

In [44]:
#Aplicar o describer para ver as estatísticas dos atributos do tipo texto
print('\nEstatísticas descritivas dos atributos do tipo texto:\n')
df_datasus.describe(include='object')


Estatísticas descritivas dos atributos do tipo texto:



,estado,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O
count,58634,56606,58634,58407,58619,57298,54305,45162,58634,58634,58463,40872,58634,58544
unique,27,4,1826,22200,2,5,5,5,1108,4769,6,2,301,527
top,SP,Suicídio,01-01-17,15-06-81,Masculino,Branca,Solteiro,8 a 11 anos,0,São Paulo,Domicílio,Não,X700,X700
freq,11256,55484,64,12,46169,29193,29408,14503,12353,2090,36070,32629,28997,26666


In [45]:
df_datasus.groupby(['estado']).size().sort_values(ascending=False)

estado
SP    11256
MG     7017
RS     6023
PR     3811
SC     3378
CE     3029
RJ     2944
BA     2655
GO     2377
PE     1906
PI     1484
MA     1466
PA     1408
MS     1184
AM     1132
PB     1048
ES      983
MT      905
RN      883
DF      779
SE      608
AL      587
RO      536
TO      524
AC      267
AP      231
RR      213
dtype: int64

In [46]:
df_datasus.groupby(['CIRCOBITO']).size().sort_values(ascending=False)

CIRCOBITO
Suicídio     55484
Acidente      1050
Homicídio       47
Outro           25
dtype: int64

In [47]:
df_datasus.groupby(['SEXO']).size().sort_values(ascending=False)

SEXO
Masculino    46169
Feminino     12450
dtype: int64

In [48]:
df_datasus.groupby(['RACACOR']).size().sort_values(ascending=False)

RACACOR
Branca      29193
Parda       24154
Preta        3050
Indígena      658
Amarela       243
dtype: int64

In [49]:
df_datasus.groupby(['LOCOCOR']).size().sort_values(ascending=False)

LOCOCOR
Domicílio                         36070
Outros                             9277
Hospital                           8577
Via pública                        3564
Outro estabelecimento de saúde      951
6                                    24
dtype: int64

### 3.1 Selecionar Colunas Relevantes

In [50]:
print('\n Colunas disponíveis no DataFrame:')
df_datasus.columns


 Colunas disponíveis no DataFrame:


Index(['estado', 'ano', 'CIRCOBITO', 'DTOBITO', 'DTNASC', 'SEXO', 'RACACOR',
       'ESTCIV', 'ESC', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'ASSISTMED',
       'CAUSABAS', 'CAUSABAS_O', 'idade', 'mes'],
      dtype='object')

In [51]:
# Pegando somente as informações da categoria Suicídio: 
df_datasus = df_datasus[df_datasus['CIRCOBITO'] == 'Suicídio']

df_datasus.head()

,estado,ano,CIRCOBITO,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,CODMUNRES,LOCOCOR,ASSISTMED,CAUSABAS,CAUSABAS_O,idade,mes
0,AC,2014,Suicídio,02-01-14,02-07-77,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Rio Branco,Domicílio,Não,X700,X700,37.0,1
1,AC,2014,Suicídio,23-01-14,30-07-66,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Rio Branco,Domicílio,Não,X800,X800,48.0,1
2,AC,2014,Suicídio,31-01-14,28-07-43,Masculino,Branca,NaN,NaN,0,Rio Branco,Domicílio,Não,X700,X700,71.0,1
3,AC,2014,Suicídio,05-02-14,04-07-99,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Epitaciolândia,Outros,Não,X700,X700,15.0,2
4,AC,2014,Suicídio,06-02-14,02-05-72,Masculino,Branca,NaN,NaN,0,Rio Branco,Outros,Não,X700,X700,42.0,2


In [52]:
lista_colunas = [
     'DTOBITO',
    'DTNASC', 
    'SEXO', 
    'RACACOR',
    'ESTCIV', 
    'ESC', 
    'OCUP',
    'LOCOCOR',
    'idade'
]

print(f'\nColunas selecionadas:\n {lista_colunas}')

#Pegando as informações de df_datasus, nas colunas escolhidas:
df_datasus_selecionados = df_datasus[lista_colunas]

print(f'\nCinco primeiras linhas da base de dados df_datasus_selecionados:\n')
df_datasus_selecionados.head()


Colunas selecionadas:
 ['DTOBITO', 'DTNASC', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'OCUP', 'LOCOCOR', 'idade']

Cinco primeiras linhas da base de dados df_datasus_selecionados:



,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,LOCOCOR,idade
0,02-01-14,02-07-77,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,37.0
1,23-01-14,30-07-66,Masculino,Branca,União consensual,12 e mais,MEDICO GINECOLOGISTA E OBSTETRA,Domicílio,48.0
2,31-01-14,28-07-43,Masculino,Branca,NaN,NaN,0,Domicílio,71.0
3,05-02-14,04-07-99,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15.0
4,06-02-14,02-05-72,Masculino,Branca,NaN,NaN,0,Outros,42.0


In [53]:
#Colunas de registros vazios (nulos):
df_datasus_selecionados.isnull().sum()

DTOBITO        0
DTNASC       208
SEXO          14
RACACOR     1231
ESTCIV      4048
ESC        12813
OCUP           0
LOCOCOR      147
idade        208
dtype: int64

In [54]:
df_datasus_selecionados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55484 entries, 0 to 58633
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DTOBITO  55484 non-null  object 
 1   DTNASC   55276 non-null  object 
 2   SEXO     55470 non-null  object 
 3   RACACOR  54253 non-null  object 
 4   ESTCIV   51436 non-null  object 
 5   ESC      42671 non-null  object 
 6   OCUP     55484 non-null  object 
 7   LOCOCOR  55337 non-null  object 
 8   idade    55276 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.2+ MB


In [55]:
# excluindo as linhas com valores ausentes
df_datasus_selecionados = df_datasus_selecionados.dropna(subset=['DTNASC'])

In [56]:
# Convertrr para o tipo datatime as colunas de data

#Transformando a coluna DTOBITO para o tipo DateTime
df_datasus_selecionados['DTOBITO'] = pd.to_datetime(df_datasus_selecionados['DTOBITO'])

#Transformando a coluna DTNASC para o tipo DateTime
df_datasus_selecionados['DTNASC'] = pd.to_datetime(df_datasus_selecionados['DTNASC'])

#### Vou desconciderar obitos de 10 anos de idade

In [57]:
data_inicio = '2008-01-01'
data_fim = '2018-12-30'
#mostra a base selecionando os elementos com o intervalo de data_inicio e data_fim
print(df_datasus_selecionados[(df_datasus_selecionados['DTNASC']>= data_inicio) & (df_datasus_selecionados['DTNASC']<= data_fim)])
#Retorna a quantidad de registro com os intervalos data_inicio e data_fim
print(
        'Tamanho: ',
            len(df_datasus_selecionados[
            (df_datasus_selecionados['DTNASC'] >= data_inicio) & 
            (df_datasus_selecionados['DTNASC'] <= data_fim)]
    ))

         DTOBITO     DTNASC       SEXO   RACACOR    ESTCIV          ESC  \
2731  2014-06-14 2013-08-23  Masculino   Amarela       NaN   1 a 3 anos   
3784  2014-09-16 2016-02-22  Masculino     Parda     Viúvo   4 a 7 anos   
10528 2014-06-17 2016-04-19   Feminino    Branca  Solteiro      Nenhuma   
11245 2015-12-02 2011-04-15  Masculino     Parda  Solteiro      Nenhuma   
12614 2015-02-14 2018-05-23  Masculino       NaN       NaN   4 a 7 anos   
14188 2015-04-17 2015-01-06  Masculino    Branca     Viúvo          NaN   
16445 2015-01-12 2015-09-24  Masculino     Parda  Solteiro   4 a 7 anos   
18021 2015-04-22 2018-01-07   Feminino    Branca     Viúvo          NaN   
21109 2015-05-05 2018-01-14  Masculino     Parda  Solteiro      Nenhuma   
21961 2016-05-02 2016-01-21  Masculino     Parda       NaN          NaN   
23518 2016-08-31 2016-02-10   Feminino    Branca  Solteiro  8 a 11 anos   
24695 2016-12-20 2018-06-11  Masculino    Branca     Viúvo   1 a 3 anos   
28866 2016-05-09 2018-12-

In [58]:
print('Tamanho Original do DF(DataFrame): ',len(df_datasus_selecionados))
exclucao_datas = ((df_datasus_selecionados['DTNASC'] >= data_inicio) & (df_datasus_selecionados['DTNASC'] <= data_fim))
df_datasus_selecionados = df_datasus_selecionados[~exclucao_datas]
print('Tamanho pós exclusão de intervalos do DF(DataFrame): ',len(df_datasus_selecionados))

Tamanho Original do DF(DataFrame):  55276
Tamanho pós exclusão de intervalos do DF(DataFrame):  55244


In [59]:
####Area destinada a remoção de datas que estão negativas em que dtobito é menor que dtnasc ex: obito: 1930-01-01 e nascimento: 2000-01-01
print('Tamanho Original do DF(DataFrame): ',len(df_datasus_selecionados))
condicao_datas_invalidas = df_datasus_selecionados['DTOBITO'] < df_datasus_selecionados['DTNASC']

# Remova as linhas que satisfazem essa condição inválida
df_datasus_selecionados = df_datasus_selecionados[~condicao_datas_invalidas].copy()

print(f"Tamanho do DF após remover registros com DTOBITO < DTNASC: {len(df_datasus_selecionados)}")

Tamanho Original do DF(DataFrame):  55244
Tamanho do DF após remover registros com DTOBITO < DTNASC: 28739


In [60]:
## Excluindo Óbitos de Menores de 11 Anos

# Calcular a idade com precisão:
# Primeiro, a diferença de ano bruta
df_datasus_selecionados['IDADE_BRUTA_ANOS'] = df_datasus_selecionados['DTOBITO'].dt.year - df_datasus_selecionados['DTNASC'].dt.year

# Ajustar para o caso do aniversário não ter chegado ainda no ano do óbito
condicao_aniversario_nao_chegou = (
    (df_datasus_selecionados['DTOBITO'].dt.month < df_datasus_selecionados['DTNASC'].dt.month) |
    ((df_datasus_selecionados['DTOBITO'].dt.month == df_datasus_selecionados['DTNASC'].dt.month) & 
     (df_datasus_selecionados['DTOBITO'].dt.day < df_datasus_selecionados['DTNASC'].dt.day))
)


# Subtrair 1 ano da idade bruta para aqueles cujo aniversário ainda não ocorreu no ano do óbito
df_datasus_selecionados.loc[condicao_aniversario_nao_chegou, 'IDADE_BRUTA_ANOS'] -= 1

# Renomear para clareza
df_datasus_selecionados.rename(columns={'IDADE_BRUTA_ANOS': 'IDADE_NO_OBITO'}, inplace=True)

# Agora, crie a condição para excluir os óbitos de pessoas com menos de 11 anos
# (Ou seja, IDADE_NO_OBITO < 11)
condicao_excluir_idade_menor_11 = (df_datasus_selecionados['IDADE_NO_OBITO'] < 11)

# Aplicar a condição para manter apenas quem tem 11 anos ou mais
df_datasus_selecionados = df_datasus_selecionados[~condicao_excluir_idade_menor_11].copy()

# Opcional: remover a coluna de idade temporária se não for mais necessária
df_datasus_selecionados = df_datasus_selecionados.drop(columns=['idade'])

# Redefinir o índice
df_datasus_selecionados = df_datasus_selecionados.reset_index(drop=True)

print(f"Tamanho do DF após exclusão de óbitos < 11 anos: {len(df_datasus_selecionados)}")

print('\nInformações do dataframe:')
df_datasus_selecionados.info()

print('\nHead do dataframe:\n')
df_datasus_selecionados.head()

Tamanho do DF após exclusão de óbitos < 11 anos: 28718

Informações do dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28718 entries, 0 to 28717
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DTOBITO         28718 non-null  datetime64[ns]
 1   DTNASC          28718 non-null  datetime64[ns]
 2   SEXO            28715 non-null  object        
 3   RACACOR         28114 non-null  object        
 4   ESTCIV          26873 non-null  object        
 5   ESC             22691 non-null  object        
 6   OCUP            28718 non-null  object        
 7   LOCOCOR         28635 non-null  object        
 8   IDADE_NO_OBITO  28718 non-null  int32         
dtypes: datetime64[ns](2), int32(1), object(6)
memory usage: 1.9+ MB

Head do dataframe:



,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,LOCOCOR,IDADE_NO_OBITO
0,2014-02-01,1977-02-07,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,36
1,2014-05-02,1999-04-07,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15
2,2014-02-27,1989-06-08,Masculino,Parda,Solteiro,1 a 3 anos,0,Outros,24
3,2014-10-03,1975-02-28,Masculino,Branca,Casado,12 e mais,0,Domicílio,39
4,2014-03-24,1977-09-10,Feminino,Parda,Solteiro,8 a 11 anos,DONA DE CASA,Domicílio,36


In [64]:
#Contando os valores da coluna Sexo considerando NaN também
print('\nContando os valores da coluna Sexo considerando NaN:\n')
df_datasus_selecionados['SEXO'].value_counts(dropna=False)


Contando os valores da coluna Sexo considerando NaN:



SEXO
Masculino        22584
Feminino          6131
Não Informado        3
Name: count, dtype: int64

In [62]:
#Tratando o campo de sexo
df_datasus_selecionados['SEXO'].fillna('Não Informado', inplace=True)

In [63]:
df_datasus_selecionados['RACACOR'].fillna('Não Informado',inplace=True)
df_datasus_selecionados['ESTCIV'].fillna('Não Informado',inplace=True)
df_datasus_selecionados['ESC'].fillna('Não Informado',inplace=True)
df_datasus_selecionados['LOCOCOR'].fillna('Não Informado',inplace=True)

In [65]:
df_datasus_selecionados.isnull().sum()

DTOBITO           0
DTNASC            0
SEXO              0
RACACOR           0
ESTCIV            0
ESC               0
OCUP              0
LOCOCOR           0
IDADE_NO_OBITO    0
dtype: int64

In [66]:
###Convertendo os números 0 em 'Não Informado'
#Total de Campos '0'
print('\nTotal de Campos "0" na coluna OCUP:')
print((df_datasus_selecionados['OCUP'] == '0').sum())

#Convertendo campo '0' para 'Não Informado':
df_datasus_selecionados.loc[df_datasus_selecionados['OCUP'] == '0', 'OCUP'] = 'Não Informado'

print('\nTotal de Campos "0" na coluna OCUP:')
print((df_datasus_selecionados['OCUP'] == '0').sum())


Total de Campos "0" na coluna OCUP:
6355

Total de Campos "0" na coluna OCUP:
0


In [67]:
df_datasus_selecionados.head(20)

,DTOBITO,DTNASC,SEXO,RACACOR,ESTCIV,ESC,OCUP,LOCOCOR,IDADE_NO_OBITO
0,2014-02-01,1977-02-07,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,36
1,2014-05-02,1999-04-07,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15
2,2014-02-27,1989-06-08,Masculino,Parda,Solteiro,1 a 3 anos,Não Informado,Outros,24
3,2014-10-03,1975-02-28,Masculino,Branca,Casado,12 e mais,Não Informado,Domicílio,39
4,2014-03-24,1977-09-10,Feminino,Parda,Solteiro,8 a 11 anos,DONA DE CASA,Domicílio,36
5,2014-04-29,1995-09-13,Masculino,Parda,Solteiro,8 a 11 anos,ESTUDANTE,Outros,18
6,2014-05-15,1995-05-10,Masculino,Branca,Solteiro,8 a 11 anos,ESTUDANTE,Outros,19
7,2014-05-24,1988-10-04,Masculino,Parda,Solteiro,4 a 7 anos,Não Informado,Domicílio,25
8,2014-05-25,1977-08-09,Masculino,Parda,Solteiro,1 a 3 anos,Não Informado,Domicílio,36
9,2014-05-06,1979-10-10,Masculino,Preta,Solteiro,8 a 11 anos,PRODUTOR AGRICOLA POLIVALENTE,Domicílio,34


### 3.3 Renomeando Colunas

In [68]:
df_datasus_selecionados.rename(columns={
    'DTOBITO':'data_obito',
    'DTNASC':'data_nascimento',
    'SEXO': 'sexo',
    'RACACOR':'raca_cor',
    'ESTCIV': 'estado_civil',
    'ESC': 'escolaridade',
    'OCUP':'ocupacao',
    'LOCOCOR': 'local_ocorrencia',
    'IDADE_NO_OBITO': 'idade_obito'
}, inplace=True)

df_datasus_selecionados.head()

,data_obito,data_nascimento,sexo,raca_cor,estado_civil,escolaridade,ocupacao,local_ocorrencia,idade_obito
0,2014-02-01,1977-02-07,Masculino,Preta,Casado,4 a 7 anos,ATLETA PROFISSIONAL DE FUTEBOL,Domicílio,36
1,2014-05-02,1999-04-07,Masculino,Branca,Solteiro,4 a 7 anos,ESTUDANTE,Outros,15
2,2014-02-27,1989-06-08,Masculino,Parda,Solteiro,1 a 3 anos,Não Informado,Outros,24
3,2014-10-03,1975-02-28,Masculino,Branca,Casado,12 e mais,Não Informado,Domicílio,39
4,2014-03-24,1977-09-10,Feminino,Parda,Solteiro,8 a 11 anos,DONA DE CASA,Domicílio,36


In [69]:
df_datasus_selecionados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28718 entries, 0 to 28717
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data_obito        28718 non-null  datetime64[ns]
 1   data_nascimento   28718 non-null  datetime64[ns]
 2   sexo              28718 non-null  object        
 3   raca_cor          28718 non-null  object        
 4   estado_civil      28718 non-null  object        
 5   escolaridade      28718 non-null  object        
 6   ocupacao          28718 non-null  object        
 7   local_ocorrencia  28718 non-null  object        
 8   idade_obito       28718 non-null  int32         
dtypes: datetime64[ns](2), int32(1), object(6)
memory usage: 1.9+ MB


In [ ]:
# #Convertendo campo idade de float para int:
# df_datasus_selecionados['idade'] = df_datasus_selecionados['idade'].astype(int)

# df_datasus_selecionados.head()

In [70]:
#Salvando em um arquivo CSV
df_datasus_selecionados.to_csv('arquivo_tratado_datasus.csv', encoding='ISO-8859-1', index=False, sep=',')

# Passo 4: L (LOAD) - Carregamento no banco de dados

### 4.1 Conectando com o SGBD (Postgres):

In [5]:
#configurando banco de dados
db_user = 'postgres'
db_password = '1234'
db_host = 'localhost'
db_database = 'db_datasus'
db_porta = '5432'


In [6]:
# Criando conexão para o SQLAlchemy
try:
    db_connection_str = f'postgresql://{db_password}:{db_user}:@{db_host}:{db_porta}/{db_database}?client_encoding=ISO-8859-1'
    db_connection = create_engine(db_connection_str)
except Exception as e:
    print('Erro:',{e})

In [ ]:
#criando banco de dados com o nome informado no create_engine:
# create_database(db_connection)

In [1]:
#conexão com o banco de dados
try:
    connection = psycopg2.connect(host=db_host,database=db_database,user=db_user,password=db_password)
    connection.autocommit = True
    cursor = connection.cursor()
except Exception as e:
    print('Erro:',{e})

Erro: {NameError("name 'psycopg2' is not defined")}


In [94]:
#Criando tabele datasus no banco de dados:
cursor.execute('CREATE TABLE public.datasus( id serial primary key, data_obito date, data_nascimento date, sexo varchar, raca_cor varchar, estado_civil varchar, escolaridade varchar, ocupacao varchar, local_ocorrencia varchar, idade_obito int)')
connection.commit()
cursor.execute('SELECT * FROM public.datasus;') 
cursor.fetchall()

[]

In [92]:
# Carregando o arquivo CSV de volta para um DataFrame
df_datasus_final = pd.read_csv('arquivo_tratado_datasus.csv', encoding='ISO-8859-1',sep=',')

In [ ]:
# 3. Usando o to_sql para inserir os dados
try:
    df_datasus_final.to_sql(
        'datasus',         # Nome da tabela no banco de dados
        con=db_connection, # Conexão com o banco de dados
        schema='public',   # Esquema (geralmente 'public')
        if_exists='append', # 'append': adiciona novas linhas; 'replace': recria a tabela; 'fail': gera erro se a tabela existir
        index=False        # Não escreve o índice do DataFrame como uma coluna
    )
    print("Dados inseridos com sucesso na tabela datasus usando to_sql!")
except Exception as e:
    print(f"Erro ao inserir dados com to_sql: {e}")
finally:
    # Fechando a conexão
    db_connection.dispose()

Dados inseridos com sucesso na tabela datasus usando to_sql!


In [96]:
try:
    cur = connection.cursor() 
    # 2. Executar a consulta com a formatação
    query = """
    SELECT
        TO_CHAR(data_obito, 'DD-MM-YYYY') AS data_obito_formatada,
        TO_CHAR(data_nascimento, 'DD-MM-YYYY') AS data_nascimento_formatada,
        -- Inclua aqui todas as outras colunas que você precisa selecionar
        sexo,
        raca_cor,
        estado_civil,
        escolaridade,
        ocupacao,
        local_ocorrencia,
        idade_obito
    FROM
        datasus;
    """
    cur.execute(query)

    # 3. Recuperar os resultados (opcional, dependendo do que você quer fazer)
        # Exemplo: Imprimir as 5 primeiras linhas
    resultados = cur.fetchall()
    for i, row in enumerate(resultados):
        if i >= 5:
            break
        print(row)

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # 4. Fechar a conexão
    if connection:
        cur.close()
        connection.close()
        print("Conexão com o PostgreSQL fechada.")

('01-02-2014', '07-02-1977', 'Masculino', 'Preta', 'Casado', '4 a 7 anos', 'ATLETA PROFISSIONAL DE FUTEBOL', 'Domicílio', 36)
('02-05-2014', '07-04-1999', 'Masculino', 'Branca', 'Solteiro', '4 a 7 anos', 'ESTUDANTE', 'Outros', 15)
('27-02-2014', '08-06-1989', 'Masculino', 'Parda', 'Solteiro', '1 a 3 anos', 'Não Informado', 'Outros', 24)
('03-10-2014', '28-02-1975', 'Masculino', 'Branca', 'Casado', '12 e mais', 'Não Informado', 'Domicílio', 39)
('24-03-2014', '10-09-1977', 'Feminino', 'Parda', 'Solteiro', '8 a 11 anos', 'DONA DE CASA', 'Domicílio', 36)
Conexão com o PostgreSQL fechada.


In [8]:
#Criando schema:
try:
    connection.commit()
    cursor.execute('create schema dw_datasus_suicidios')
except Exception as e:
    print('Erro "criando schema":',{e})


In [ ]:
try:
    connection.commit()
    cursor.execute('create table dw_datasus_suicidios.pessoa(id_pessoa serial primary key, data_nascimento date, sexo varchar, raca_cor varchar, estado_civil varchar, escolaridade varchar, ocupacao varchar)')
    print('Criado tabela Pessoa com sucesso!')
except Exception as e:
    print('Erro "criando tabela pessoa":',{e})

In [ ]:
try:
    connection.commit()
    cursor.execute('insert into dw_datasus_suicidios.pessoa(data_nascimento,sexo,raca_cor,estado_civil,escolaridade,ocupacao)'
    'select distinct data_nascimento, sexo, raca_cor, estado_civil, escolaridade,ocupacao from public.datasus')

except Exception as e:
    print('Erro Insert tabela Pessoa:',{e})

Erro select: {ProgrammingError('no results to fetch')}


In [16]:
try:
    cursor.execute(
    'select * from dw_datasus_suicidios.pessoa')
    resultado = cursor.fetchall() 
    for i, row in enumerate(resultado):
        if i >= 5:
            break
        print(row)
except Exception as e:
    print('Erro select:',{e})
finally:
    # 4. Fechar a conexão
    if connection:
        cursor.close()
        connection.close()
        print("Conexão com o PostgreSQL fechada.")

(1, datetime.date(1983, 6, 8), 'Masculino', 'Parda', 'Solteiro', '4 a 7 anos', 'PRODUTOR AGROPECUARIO, EM GERAL')
(2, datetime.date(1995, 1, 27), 'Masculino', 'Branca', 'Solteiro', '8 a 11 anos', 'ESTUDANTE')
(3, datetime.date(1982, 4, 27), 'Masculino', 'Parda', 'Não Informado', 'Não Informado', 'Não Informado')
(4, datetime.date(1986, 2, 28), 'Feminino', 'Branca', 'Solteiro', '8 a 11 anos', 'COMERCIANTE VAREJISTA')
(5, datetime.date(1986, 5, 2), 'Masculino', 'Branca', 'Solteiro', '8 a 11 anos', 'VENDEDOR DE COMERCIO VAREJISTA')
Conexão com o PostgreSQL fechada.


In [24]:
try:
    connection.commit()
    cursor.execute('create table dw_datasus_suicidios.fato_suicidio(id serial primary key, ' \
    'data_obito date, ' \
    'id_pessoa int, ' \
    'constraint fk_d_pessoa foreign key (id_pessoa) references dw_datasus_suicidios.pessoa(id_pessoa));')
    print('Criado tabela fato_suicidio com sucesso!')
except Exception as e:
    print('Erro "criando tabela pessoa":',{e})
finally:
    # 4. Fechar a conexão
    if connection:
        cursor.close()
        connection.close()
        print("Conexão com o PostgreSQL fechada.")

Criado tabela fato_suicidio com sucesso!
Conexão com o PostgreSQL fechada.


In [ ]:
try:
    connection.commit()
    cursor.execute('insert into dw_datasus_suicidios.fato_suicidio(data_obito,id_pessoa)' \
    'select dt.data_obito, p.id_pessoa from public.datasus as dt ' \
    'inner join dw_datasus_suicidios.pessoa as p on p.data_nascimento = dt.data_nascimento and ' \
    'p.sexo = dt.sexo and ' \
    'p.raca_cor = dt.raca_cor and ' \
    'p.estado_civil = dt.estado_civil and ' \
    'p.escolaridade = dt.escolaridade and ' \
    'p.ocupacao = dt.ocupacao')
    print('Fato_suicidio inserido com sucesso!')
except Exception as e:
    print('Erro "insert tabela fato_sucidio":',{e})
# finally:
#     # 4. Fechar a conexão
#     # if connection:
#     #     cursor.close()
#     #     connection.close()
#     #     print("Conexão com o PostgreSQL fechada.")

Fato_suicidio


In [35]:
try:
    connection.commit()
    cursor.execute('select * from dw_datasus_suicidios.fato_suicidio')
    resultado = cursor.fetchall() 
    for i, row in enumerate(resultado):
        if i >= 5:
            break
        print(row)
except Exception as e:
    print('Erro "select tabela fato_suicidio":',{e})
finally:
    # 4. Fechar a conexão
    if connection:
        cursor.close()
        connection.close()
        print("Conexão com o PostgreSQL fechada.")

(1, datetime.date(2014, 2, 1), 1045)
(2, datetime.date(2014, 5, 2), 1950)
(3, datetime.date(2014, 2, 27), 28980)
(4, datetime.date(2014, 10, 3), 11214)
(5, datetime.date(2014, 3, 24), 30071)
Conexão com o PostgreSQL fechada.
